In [1]:
# Add the dependencies.
import pandas as pd
import os
from sqlalchemy import create_engine, func
from config import DATABASE_URI
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from tensorflow.keras import metrics
#from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#import sqlalchemy
#get data from postgres

engine = create_engine(DATABASE_URI)
CSTEMP_df = pd.read_sql_table(
    'ec1',
    con=engine
)
CSTEMP_df.head()

,idd,t1,pr1,f1,f2,f3,o_o,pw1,re,x1,x2,x1r,x2r,z,a1
0,C2,17,0.0645,100.0,100.0,55.0,False,400.0,0.0,0.0,0.0,85.0,10.0,1.54 - 9.28j,0.0
1,C2,123,0.0645,100.0,100.0,55.0,False,400.0,0.0,0.0,0.0,85.0,10.0,1.54 - 9.28j,0.0
2,C2,232,0.0645,100.0,100.0,55.0,False,400.0,0.0,0.0,0.0,85.0,10.0,1.54 - 9.28j,0.0
3,C2,346,0.0645,100.0,100.0,55.0,False,400.0,0.0,0.0,0.0,85.0,10.0,1.54 - 9.28j,0.0
4,C2,454,0.0645,100.0,100.0,55.0,False,400.0,0.0,0.0,0.0,85.0,10.0,1.54 - 9.28j,0.0


In [3]:
temp1_df = CSTEMP_df[CSTEMP_df['o_o'] == True]
temp1_df['pw1_Diff'] = temp1_df['pw1'].diff()
temp1_df['index1'] = temp1_df.index
temp1_df.head()

C:\Users\laksh\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\laksh\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,idd,t1,pr1,f1,f2,f3,o_o,pw1,re,x1,x2,x1r,x2r,z,a1,pw1_Diff,index1
67,C2,7343,0.0645,100.0,100.0,55.0,True,0.0,0.0,85.0,10.0,85.0,10.0,1.54 - 9.28j,0.0,NaN,67
68,C2,7452,0.0645,100.0,100.0,55.0,True,0.0,0.0,85.0,10.0,85.0,10.0,1.54 - 9.28j,0.0,0.0,68
69,C2,7565,0.0645,100.0,100.0,55.0,True,0.0,0.0,85.0,10.0,85.0,10.0,1.54 - 9.28j,0.0,0.0,69
70,C2,7674,0.0645,100.0,100.0,55.0,True,0.0,0.0,85.0,10.0,85.0,10.0,1.54 - 9.28j,0.0,0.0,70
71,C2,7780,0.0645,100.0,100.0,55.0,True,0.0,0.0,85.0,10.0,85.0,10.0,1.54 - 9.28j,0.0,0.0,71


In [4]:
temp2_df = temp1_df[(temp1_df['pw1'] > 0) & \
                    ((temp1_df['re'] > temp1_df['pw1']*0.02 )|(temp1_df['re'] == 0)) & (temp1_df['pr1'] > 0.017)]
temp2_df.tail()

,idd,t1,pr1,f1,f2,f3,o_o,pw1,re,x1,x2,x1r,x2r,z,a1,pw1_Diff,index1
28650,C2,3133864,0.0353,50.0,50.0,95.0,True,400.0,164.644,85.0,10.0,34.4,27.5,7.48 - 61.9j,410.0,0.0,28650
28651,C2,3133991,0.0354,50.0,50.0,95.0,True,400.0,142.464,85.0,10.0,49.1,25.0,7.48 - 61.9j,410.0,0.0,28651
28652,C2,3134092,0.0354,50.0,50.0,95.0,True,400.0,33.172,85.0,10.0,49.1,25.0,4.18 - 60.0j,552.8,0.0,28652
28653,C2,3134191,0.0354,50.0,50.0,95.0,True,400.0,35.088,85.0,10.0,49.1,25.0,4.18 - 60.0j,552.8,0.0,28653
28654,C2,3134297,0.0355,50.0,50.0,95.0,True,400.0,35.088,85.0,10.0,67.4,22.8,2.58 - 59.8j,552.8,0.0,28654


In [5]:
temp2_df['index1_Diff'] = temp2_df['index1'] - temp2_df['index1'].shift(-1)
temp2_df.tail()

C:\Users\laksh\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,idd,t1,pr1,f1,f2,f3,o_o,pw1,re,x1,x2,x1r,x2r,z,a1,pw1_Diff,index1,index1_Diff
28650,C2,3133864,0.0353,50.0,50.0,95.0,True,400.0,164.644,85.0,10.0,34.4,27.5,7.48 - 61.9j,410.0,0.0,28650,-1.0
28651,C2,3133991,0.0354,50.0,50.0,95.0,True,400.0,142.464,85.0,10.0,49.1,25.0,7.48 - 61.9j,410.0,0.0,28651,-1.0
28652,C2,3134092,0.0354,50.0,50.0,95.0,True,400.0,33.172,85.0,10.0,49.1,25.0,4.18 - 60.0j,552.8,0.0,28652,-1.0
28653,C2,3134191,0.0354,50.0,50.0,95.0,True,400.0,35.088,85.0,10.0,49.1,25.0,4.18 - 60.0j,552.8,0.0,28653,-1.0
28654,C2,3134297,0.0355,50.0,50.0,95.0,True,400.0,35.088,85.0,10.0,67.4,22.8,2.58 - 59.8j,552.8,0.0,28654,NaN


In [7]:
temp3_df = temp2_df[((temp2_df['pw1_Diff'] > 0) & (temp2_df['index1_Diff'] == -1))|(temp2_df['index1_Diff'] <-4) ]
temp3_df.tail()

,idd,t1,pr1,f1,f2,f3,o_o,pw1,re,x1,x2,x1r,x2r,z,a1,pw1_Diff,index1,index1_Diff
28457,C2,3112759,0.0354,50.0,50.0,95.0,True,400.0,197.324,65.0,20.0,65.0,20.0,2.75 - 51.0j,0.0,400.0,28457,-1.0
28461,C2,3113200,0.0354,50.0,50.0,95.0,True,400.0,21.224,65.0,20.0,58.0,22.2,3.37 - 59.0j,583.4,0.0,28461,-87.0
28548,C2,3122710,0.0354,50.0,50.0,95.0,True,400.0,0.000,75.0,15.0,75.0,15.0,2.05 - 34.7j,0.0,400.0,28548,-1.0
28559,C2,3123902,0.0355,50.0,50.0,95.0,True,400.0,14.428,75.0,15.0,59.1,23.2,2.89 - 59.8j,588.0,0.0,28559,-81.0
28640,C2,3132772,0.0353,50.0,50.0,95.0,True,400.0,199.992,85.0,10.0,85.0,10.0,1.54 - 9.28j,0.0,400.0,28640,-1.0


In [9]:
temp3_df['time'] = temp3_df['t1'].diff()
temp3_df.tail()

C:\Users\laksh\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,idd,t1,pr1,f1,f2,f3,o_o,pw1,re,x1,x2,x1r,x2r,z,a1,pw1_Diff,index1,index1_Diff,time
28457,C2,3112759,0.0354,50.0,50.0,95.0,True,400.0,197.324,65.0,20.0,65.0,20.0,2.75 - 51.0j,0.0,400.0,28457,-1.0,9304.0
28461,C2,3113200,0.0354,50.0,50.0,95.0,True,400.0,21.224,65.0,20.0,58.0,22.2,3.37 - 59.0j,583.4,0.0,28461,-87.0,441.0
28548,C2,3122710,0.0354,50.0,50.0,95.0,True,400.0,0.000,75.0,15.0,75.0,15.0,2.05 - 34.7j,0.0,400.0,28548,-1.0,9510.0
28559,C2,3123902,0.0355,50.0,50.0,95.0,True,400.0,14.428,75.0,15.0,59.1,23.2,2.89 - 59.8j,588.0,0.0,28559,-81.0,1192.0
28640,C2,3132772,0.0353,50.0,50.0,95.0,True,400.0,199.992,85.0,10.0,85.0,10.0,1.54 - 9.28j,0.0,400.0,28640,-1.0,8870.0


In [11]:
temp3_df.drop(temp3_df.tail(1).index,inplace=True) # drop last n rows
temp3_df.tail()

C:\Users\laksh\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,idd,t1,pr1,f1,f2,f3,o_o,pw1,re,x1,x2,x1r,x2r,z,a1,pw1_Diff,index1,index1_Diff,time
28372,C2,3103455,0.0355,50.0,50.0,95.0,True,400.0,0.000,55.0,25.0,74.9,22.2,2.10 - 59.7j,595.2,0.0,28372,-85.0,758.0
28457,C2,3112759,0.0354,50.0,50.0,95.0,True,400.0,197.324,65.0,20.0,65.0,20.0,2.75 - 51.0j,0.0,400.0,28457,-1.0,9304.0
28461,C2,3113200,0.0354,50.0,50.0,95.0,True,400.0,21.224,65.0,20.0,58.0,22.2,3.37 - 59.0j,583.4,0.0,28461,-87.0,441.0
28548,C2,3122710,0.0354,50.0,50.0,95.0,True,400.0,0.000,75.0,15.0,75.0,15.0,2.05 - 34.7j,0.0,400.0,28548,-1.0,9510.0
28559,C2,3123902,0.0355,50.0,50.0,95.0,True,400.0,14.428,75.0,15.0,59.1,23.2,2.89 - 59.8j,588.0,0.0,28559,-81.0,1192.0


In [25]:
temp4_df = temp3_df[(temp3_df['index1_Diff'] < -1 ) & (temp3_df['pw1_Diff'] ==0 )]
temp4_df.head()

,idd,t1,pr1,f1,f2,f3,o_o,pw1,re,x1,x2,x1r,x2r,z,a1,pw1_Diff,index1,index1_Diff,time
215,C2,23529,0.0643,100.0,100.0,55.0,True,100.0,4.060,85.0,10.0,58.2,20.7,5.65 - 53.4j,124.6,0.0,215,-79.0,1088.0
302,C2,33270,0.0644,100.0,100.0,55.0,True,100.0,3.396,75.0,15.0,59.9,20.4,2.87 - 51.5j,125.6,0.0,302,-83.0,875.0
487,C2,53501,0.0641,100.0,100.0,55.0,True,100.0,5.204,55.0,25.0,66.8,20.2,2.53 - 51.6j,123.4,0.0,487,-81.0,1092.0
579,C2,63565,0.0646,100.0,100.0,55.0,True,100.0,7.220,45.0,30.0,48.6,21.1,4.20 - 51.3j,129.2,0.0,579,-81.0,1205.0
671,C2,73624,0.0647,100.0,100.0,55.0,True,100.0,2.724,35.0,35.0,71.8,19.8,2.31 - 51.8j,123.2,0.0,671,-80.0,1200.0


In [35]:
columnx = ['pr1','f1','f2','f3','pw1','time']
columny = ['x1','x2']

In [36]:
X = temp4_df[columnx].values
y = temp4_df[columny].values
for i in range(len(y)):
    y[i] =[y[i][0]/100,y[i][1]/100]

In [37]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [38]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 10

n1 = tf.keras.models.Sequential()

# First hidden layer
n1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
n1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
n1.add(tf.keras.layers.Dense(units=2,activation='sigmoid'))

# Check the structure of the model
n1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 20)                140       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 22        
Total params: 372
Trainable params: 372
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
n1.compile(loss='mse', optimizer="SGD",metrics=["mse"])
# Train the model
fit_model = n1.fit(X_train_scaled,y_train,batch_size = 10,epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = n1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, accuracy: {1-model_accuracy}")

Epoch 1/50
17/17 [==============================] - 0s 1ms/step - loss: 0.0513 - mse: 0.0513
Epoch 2/50
17/17 [==============================] - 0s 2ms/step - loss: 0.0500 - mse: 0.0500
Epoch 3/50
17/17 [==============================] - 0s 2ms/step - loss: 0.0486 - mse: 0.0486
Epoch 4/50
17/17 [==============================] - 0s 2ms/step - loss: 0.0474 - mse: 0.0474
Epoch 5/50
17/17 [==============================] - 0s 2ms/step - loss: 0.0463 - mse: 0.0463
Epoch 6/50
17/17 [==============================] - 0s 2ms/step - loss: 0.0452 - mse: 0.0452
Epoch 7/50
17/17 [==============================] - 0s 2ms/step - loss: 0.0441 - mse: 0.0441
Epoch 8/50
17/17 [==============================] - 0s 2ms/step - loss: 0.0432 - mse: 0.0432
Epoch 9/50
17/17 [==============================] - 0s 2ms/step - loss: 0.0422 - mse: 0.0422
Epoch 10/50
17/17 [==============================] - 0s 2ms/step - loss: 0.0413 - mse: 0.0413
Epoch 11/50
17/17 [==============================] - 0s 2ms/step - lo

In [41]:
X_scaled = X_scaler.transform(X)

In [42]:
output = n1.predict(X_scaled)

In [43]:
temp4_df["PO11"] = [i[0]*100 for i in output]
temp4_df["PO22"] = [i[1]*100 for i in output]
temp4_df

C:\Users\laksh\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\laksh\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,idd,t1,pr1,f1,f2,f3,o_o,pw1,re,x1,...,x1r,x2r,z,a1,pw1_Diff,index1,index1_Diff,time,PO11,PO22
215,C2,23529,0.0643,100.0,100.0,55.0,True,100.0,4.060,85.0,...,58.2,20.7,5.65 - 53.4j,124.6,0.0,215,-79.0,1088.0,59.510279,30.799359
302,C2,33270,0.0644,100.0,100.0,55.0,True,100.0,3.396,75.0,...,59.9,20.4,2.87 - 51.5j,125.6,0.0,302,-83.0,875.0,60.271978,30.372238
487,C2,53501,0.0641,100.0,100.0,55.0,True,100.0,5.204,55.0,...,66.8,20.2,2.53 - 51.6j,123.4,0.0,487,-81.0,1092.0,59.543806,30.796349
579,C2,63565,0.0646,100.0,100.0,55.0,True,100.0,7.220,45.0,...,48.6,21.1,4.20 - 51.3j,129.2,0.0,579,-81.0,1205.0,59.003794,31.055301
671,C2,73624,0.0647,100.0,100.0,55.0,True,100.0,2.724,35.0,...,71.8,19.8,2.31 - 51.8j,123.2,0.0,671,-80.0,1200.0,58.998108,31.050670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28192,C2,3083772,0.0356,50.0,50.0,95.0,True,400.0,11.736,35.0,...,66.7,22.8,2.50 - 59.8j,598.4,0.0,28192,-82.0,1096.0,62.906802,34.770685
28282,C2,3093620,0.0356,50.0,50.0,95.0,True,400.0,22.324,45.0,...,71.5,22.7,4.62 - 61.0j,514.0,0.0,28282,-83.0,876.0,63.995874,34.945399
28372,C2,3103455,0.0355,50.0,50.0,95.0,True,400.0,0.000,55.0,...,74.9,22.2,2.10 - 59.7j,595.2,0.0,28372,-85.0,758.0,64.754128,34.894836
28461,C2,3113200,0.0354,50.0,50.0,95.0,True,400.0,21.224,65.0,...,58.0,22.2,3.37 - 59.0j,583.4,0.0,28461,-87.0,441.0,66.756117,34.722537
